In [1]:
# Import required libraries

# Data loading and manipulation
import pandas as pd
import numpy as np

# Text preprocessing and NLP
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)



# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.colors as pc
from IPython.display import Image, display
#from wordcloud import WordCloud
from PIL import Image, ImageDraw, ImageFont

# Machine learning and preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
from imblearn.pipeline import Pipeline


# Classification algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


# Model evaluation metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

# Model interpretation and saving
import lime
import lime.lime_text
import joblib
import pickle

#webscrapping
#import tweepy
import time

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set(style="whitegrid")
sns.set_theme(rc={'figure.figsize':(11.7,8.27)})

Error importing optional module skimage
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\envs\learn-env\lib\site-packages\_plotly_utils\optional_imports.py", line 30, in get_module
    return import_module(name)
  File "c:\Users\USER\anaconda3\envs\learn-env\lib\importlib\__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 783, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "c:\Users\USER\anaconda3\envs\learn-env\lib\site-packages\skimage\__init__.py", line 122, in <module>
    from ._shared import geometry
  File "geometry.pyx", line 1

In [2]:
df = pd.read_csv('../Data/judge-1377884607_tweet_product_company.csv', encoding='ISO-8859-1')
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [4]:
df.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [5]:
df['emotion_in_tweet_is_directed_at'].fillna('unknown', inplace= True)

In [6]:
df.rename(columns={'emotion_in_tweet_is_directed_at': 'brand'}, inplace=True)

In [7]:
#df.rename(columns={'is_there_an_emotion_directed_at_a_brand_or_product': 'emotion'}, inplace=True)

In [8]:
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [9]:
def categorize_emotion(row):
    if row['is_there_an_emotion_directed_at_a_brand_or_product'] == 'No emotion toward brand or product':
        return 'neutral'
    elif row['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion':
        return 'positive'
    elif row['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion':
        return 'negative'
    else:
        return 'neutral'
    
df['emotion'] = df.apply(categorize_emotion, axis=1)

In [10]:
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive


In [11]:
df['cleaned_text'] = df['tweet_text'].str.lower().str.strip()
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative,.@wesley83 i have a 3g iphone. after 3 hrs twe...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive,@jessedee know about @fludapp ? awesome ipad/i...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive,@swonderlin can not wait for #ipad 2 also. the...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative,@sxsw i hope this year's festival isn't as cra...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive,@sxtxstate great stuff on fri #sxsw: marissa m...


In [12]:
df['cleaned_text'] = df['cleaned_text'].str.replace(r'http\S+|www\S+|@\w+|#\w+', '', regex=True)
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative,. i have a 3g iphone. after 3 hrs tweeting at ...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive,know about ? awesome ipad/iphone app that yo...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive,can not wait for 2 also. they should sale th...
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative,i hope this year's festival isn't as crashy a...
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive,"great stuff on fri : marissa mayer (google), ..."


In [13]:
df['cleaned_text'].tail()

9088                             ipad everywhere.  {link}
9089    wave, buzz... rt  we interrupt your regularly ...
9090    google's zeiger, a physician never reported po...
9091    some verizon iphone customers complained their...
9092    ï¡ïàü_êîò£áââ_£â_ûârt  ...
Name: cleaned_text, dtype: object

In [14]:

tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:'[a-z]+)?")

In [15]:
stopwords_list=set(stopwords.words('english'))
print(stopwords_list)

{"should've", 'having', 'only', 'them', 'his', 't', 've', 'be', 'that', 'which', 'at', 'above', 'whom', "needn't", "you've", 'being', 'had', "weren't", 'hers', 'yourselves', 'up', "mustn't", 'own', 'an', 'down', 'who', 'it', 'i', 'she', "we're", 'from', 'before', "she's", "shouldn't", 'further', 's', 'any', "don't", 'have', 'not', 'y', 'you', "i've", 'won', 'has', 'now', 'of', 'if', 'my', 'these', 'myself', 'off', "we'll", 'its', 'wasn', 'isn', 'does', 'are', "he'll", 'their', 'into', 'doing', 'in', "haven't", "it's", 'while', 'more', 'nor', 'on', 'until', 'wouldn', 'am', 'what', 'shouldn', 'itself', 'our', 'him', 'after', 'both', 'do', "shan't", 'can', 'out', 'mustn', 'too', "he's", 'theirs', 'here', 'other', 'he', 'herself', 'against', 'such', 'where', 'again', "that'll", "hasn't", "you'd", 'when', 'those', "she'd", 'aren', "it'd", 'your', 'all', 'why', 'themselves', 'there', 'then', 'few', 'this', 'were', 'yours', 'haven', 'for', 'they', 'as', 'been', 'but', "i'm", 'the', "we've", '

In [16]:
def remove_stopwords(text):
    if not isinstance(text, str):
        return ""
    
    tokens = tokenizer.tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stopwords_list]
    cleaned_tokens= [word for word in filtered_tokens if len(word) > 1 ]
    
    return " ".join(cleaned_tokens) 

In [17]:
df['cleaned_text'] = df['cleaned_text'].apply(remove_stopwords)
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative,iphone hrs tweeting dead need upgrade plugin s...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive,know awesome ipad iphone app likely appreciate...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive,wait also sale
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative,hope year's festival crashy year's iphone app
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive,great stuff fri marissa mayer google tim o'rei...


In [18]:
#removing punctuation-
df['cleaned_text'] = df['cleaned_text'].str.replace(r'[^A-Za-z\s]', '', regex=True)
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative,iphone hrs tweeting dead need upgrade plugin s...
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive,know awesome ipad iphone app likely appreciate...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive,wait also sale
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative,hope years festival crashy years iphone app
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive,great stuff fri marissa mayer google tim oreil...


In [19]:
df['cleaned_text'].tail()

9088                                 ipad everywhere link
9089    wave buzz rt interrupt regularly scheduled gee...
9090    googles zeiger physician never reported potent...
9091    verizon iphone customers complained time fell ...
9092                    rt google tests check offers link
Name: cleaned_text, dtype: object

In [20]:
def get_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {
        'J': wordnet.ADJ,  #Represents an Adjective
        'N': wordnet.NOUN, #Represents a Noun
        'V': wordnet.VERB, #Represents a Verb
        'R': wordnet.ADV   #Represents an Adverb
    }
    return tag_dict.get(tag, wordnet.NOUN)

In [21]:

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word, get_pos(word)) for word in tokenizer.tokenize(text)])
    
df['cleaned_text'] = df['cleaned_text'].apply(lemmatize_text)
df.head()

,tweet_text,brand,is_there_an_emotion_directed_at_a_brand_or_product,emotion,cleaned_text
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,negative,iphone hr tweet dead need upgrade plugin station
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,positive,know awesome ipad iphone app likely appreciate...
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,positive,wait also sale
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,negative,hope year festival crashy year iphone app
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,positive,great stuff fri marissa mayer google tim oreil...


In [22]:
df['cleaned_text'].tail()

9088                                 ipad everywhere link
9089    wave buzz rt interrupt regularly schedule geek...
9090    google zeiger physician never report potential...
9091    verizon iphone customer complain time fell bac...
9092                      rt google test check offer link
Name: cleaned_text, dtype: object

In [23]:
df.drop_duplicates(inplace=True)

In [24]:
df.dropna(inplace=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9070 entries, 0 to 9092
Data columns (total 5 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9070 non-null   object
 1   brand                                               9070 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9070 non-null   object
 3   emotion                                             9070 non-null   object
 4   cleaned_text                                        9070 non-null   object
dtypes: object(5)
memory usage: 425.2+ KB


In [26]:
df_cleaned = df[['cleaned_text', 'emotion']]
df_cleaned

,cleaned_text,emotion
0,iphone hr tweet dead need upgrade plugin station,negative
1,know awesome ipad iphone app likely appreciate...,positive
2,wait also sale,positive
3,hope year festival crashy year iphone app,negative
4,great stuff fri marissa mayer google tim oreil...,positive
...,...,...
9088,ipad everywhere link,positive
9089,wave buzz rt interrupt regularly schedule geek...,neutral
9090,google zeiger physician never report potential...,neutral
9091,verizon iphone customer complain time fell bac...,neutral


In [27]:
label_encoder = LabelEncoder()
df_cleaned['target'] = label_encoder.fit_transform(df_cleaned['emotion'])
df_cleaned.tail()

,cleaned_text,emotion,target
9088,ipad everywhere link,positive,2
9089,wave buzz rt interrupt regularly schedule geek...,neutral,1
9090,google zeiger physician never report potential...,neutral,1
9091,verizon iphone customer complain time fell bac...,neutral,1
9092,rt google test check offer link,neutral,1


In [28]:
df_cleaned['target'].value_counts()

1    5531
2    2970
0     569
Name: target, dtype: int64

In [29]:
X = df_cleaned['cleaned_text']
y = df_cleaned['target']

In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=42, stratify=y)


In [31]:
# Pipeline
logreg_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        max_features=5000,
        ngram_range=(1, 2)
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        solver='saga'
    ))
])

# Train
logreg_pipeline.fit(X_train, y_train)

# Predict
y_pred = logreg_pipeline.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.48217567070929807
              precision    recall  f1-score   support

           0       0.62      0.05      0.09       171
           1       0.85      0.29      0.43      1659
           2       0.39      0.93      0.54       891

    accuracy                           0.48      2721
   macro avg       0.62      0.42      0.35      2721
weighted avg       0.68      0.48      0.45      2721



In [ ]:
'''
param_grid = {
    "tfidf__max_features": [3000, 5000, 8000],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [2, 5],
    "clf__C": [0.1, 1, 3, 10],
    "clf__penalty": ["l1", "l2"]
}
grid = GridSearchCV(
    estimator=logreg_pipeline,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

print(grid.best_params_)

best_logreg_model = grid.best_estimator_

best_model = grid.best_estimator_

y_pred_tuned = best_model.predict(X_test)

print("Tuned accuracy:", accuracy_score(y_test, y_pred_tuned))
print(classification_report(y_test, y_pred_tuned))
'''




Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 32.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 64.9min finished


{'clf__C': 10, 'clf__penalty': 'l2', 'tfidf__max_features': 8000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Tuned accuracy: 0.6413083425211319
              precision    recall  f1-score   support

           0       0.26      0.45      0.33       171
           1       0.74      0.74      0.74      1659
           2       0.57      0.49      0.52       891

    accuracy                           0.64      2721
   macro avg       0.52      0.56      0.53      2721
weighted avg       0.65      0.64      0.65      2721



In [37]:
param_grid = {
    "tfidf__max_features": [5000, 8000],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [2, 5],
    "clf__C": [0.1, 1, 10],
    "clf__penalty": ["l1", "l2"]
}
grid = GridSearchCV(
    estimator=logreg_pipeline,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

print(grid.best_params_)

best_logreg_model = grid.best_estimator_

best_model = grid.best_estimator_

y_pred_tuned = best_model.predict(X_test)

print("Tuned accuracy:", accuracy_score(y_test, y_pred_tuned))
print(classification_report(y_test, y_pred_tuned))


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   37.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 30.1min finished


{'clf__C': 10, 'clf__penalty': 'l1', 'tfidf__max_features': 8000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Tuned accuracy: 0.6288129364204337
              precision    recall  f1-score   support

           0       0.28      0.45      0.34       171
           1       0.76      0.66      0.71      1659
           2       0.53      0.60      0.56       891

    accuracy                           0.63      2721
   macro avg       0.52      0.57      0.54      2721
weighted avg       0.66      0.63      0.64      2721



In [38]:
#naive bayes
nb_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])
param_grid_nb = {
    "tfidf__max_features": [3000, 5000, 8000],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [2, 5],
    "clf__alpha": [0.01, 0.1, 0.5, 1.0]
}
grid_nb = GridSearchCV(
    estimator=nb_pipeline,
    param_grid=param_grid_nb,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_nb.fit(X_train, y_train)

print(grid_nb.best_params_)
print("Best CV macro-F1:", grid_nb.best_score_)

best_nb = grid_nb.best_estimator_

y_pred_nb = best_nb.predict(X_test)

print("Tuned NB accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))



Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:   50.7s finished


{'clf__alpha': 0.01, 'tfidf__max_features': 8000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Best CV macro-F1: 0.5229929115657546
Tuned NB accuracy: 0.6629915472252849
              precision    recall  f1-score   support

           0       0.47      0.24      0.32       171
           1       0.71      0.80      0.75      1659
           2       0.57      0.49      0.53       891

    accuracy                           0.66      2721
   macro avg       0.58      0.51      0.53      2721
weighted avg       0.65      0.66      0.65      2721



In [39]:
#linear svc

svc_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", LinearSVC(class_weight="balanced"))
])
param_grid_svc = {
    "tfidf__max_features": [5000],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [2],
    "clf__C": [0.1, 1, 3]
}
grid_svc = GridSearchCV(
    svc_pipeline,
    param_grid=param_grid_svc,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=2
)

grid_svc.fit(X_train, y_train)

print(grid_svc.best_params_)
print("Best CV macro-F1:", grid_svc.best_score_)

best_svc = grid_svc.best_estimator_

y_pred_svc = best_svc.predict(X_test)

print("LinearSVC accuracy:", accuracy_score(y_test, y_pred_svc))
print(classification_report(y_test, y_pred_svc))


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    5.2s finished


{'clf__C': 1, 'tfidf__max_features': 5000, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 2)}
Best CV macro-F1: 0.5532551399925135
LinearSVC accuracy: 0.6604189636163176
              precision    recall  f1-score   support

           0       0.38      0.41      0.39       171
           1       0.75      0.74      0.74      1659
           2       0.56      0.56      0.56       891

    accuracy                           0.66      2721
   macro avg       0.56      0.57      0.57      2721
weighted avg       0.66      0.66      0.66      2721



In [ ]:
models = {
    "Logistic Regression": best_model,
    "Multinomial NB": best_nb,
    "Linear SVC": best_svc
}

print("===== Final model comparison =====\n")

for name, model in models.items():
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")
    f1_weighted = f1_score(y_test, y_pred, average="weighted")

    print(f"{name}")
    print(f"  Accuracy     : {acc:.4f}")
    print(f"  Macro F1     : {f1_macro:.4f}")
    print(f"  Weighted F1  : {f1_weighted:.4f}")
    print("-" * 35)



===== Final model comparison =====

Logistic Regression
  Accuracy     : 0.6288
  Macro F1     : 0.5391
  Weighted F1  : 0.6390
-----------------------------------
Multinomial NB
  Accuracy     : 0.6630
  Macro F1     : 0.5325
  Weighted F1  : 0.6517
-----------------------------------
Linear SVC
  Accuracy     : 0.6604
  Macro F1     : 0.5659
  Weighted F1  : 0.6615
-----------------------------------


In [ ]:
# best is linear svc because of macro F1 for imbalance
#Linear SVC achieved the highest macro-averaged F1 score (0.5659), indicating better balanced performance across all classes compared to Logistic Regression and Multinomial Naive Bayes. Therefore, Linear SVC was selected as the final model.